In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [8]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [9]:
from tensorflow.python.client import device_lib
import pickle
import math
import re

In [10]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [18]:
from nltk.corpus import stopwords
nltkStopWords = stopwords.words("german")

In [8]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16986361762654817923
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18289734811047742378
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5252283513249090627
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7592919712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15896393766569009768
physical_device_desc: "device: 0, name: GeForce RTX 2080 SUPER, pci bus id: 0000:2d:00.0, compute capability: 7.5"
]


In [8]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
export_file_path_csv = "./all_cleaned_cases.csv"
df = pd.read_csv(export_file_path_csv, sep='\t')

In [11]:
allContents = df.content.to_numpy()

In [12]:
text = ''
for i, c in enumerate(allContents):
    if(i%1000==0):
        text += str(c)

In [13]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [14]:
abbrev = ["Abb", "Abk", "allg", "eigtl", "gegr", "Nrn", "jmd", "ugs", "urspr", "usw","Aufl", "rn ", \
          "Abs", "GVG", "nr", "Art", "f", "ff", '"', "-", \
         "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", \
          "s", "t", "u", "v", "w", "x", "y", "z"]

for abb in abbrev:
    tokenizer._params.abbrev_types.add(abb.lower())
    
tokenizer._params.abbrev_types.add('vgl')
tokenizer._params.abbrev_types.add('bzw')
tokenizer._params.abbrev_types.add('e')
tokenizer._params.abbrev_types.add('o')
tokenizer._params.abbrev_types.add('s')
tokenizer._params.abbrev_types.add('bspw')
tokenizer._params.abbrev_types.add('etc')
tokenizer._params.abbrev_types.add('rn')
tokenizer._params.abbrev_types.add('vgl')
tokenizer._params.abbrev_types.add('strspr')
tokenizer._params.abbrev_types.add('bverwg')

In [25]:
type(sentences)

list

In [27]:
sentences = []

for content in allContents:
    if type(content) != type(""):
        continue
    sentences.append(tokenizer.tokenize(content))
sentences = np.array(sentences)
sentences = sentences.reshape(-1) # "flatten" the array into 1d

<ipython-input-27-8da5ab87f487>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)


In [9]:
with open("allSentences.pkl", "rb") as f:
    sentences = pickle.load(f)

In [10]:
# sentences[0]

In [11]:
retain = math.floor(len(sentences)/10)

NameError: name 'sentences' is not defined

In [12]:
smallerSents = np.random.choice(sentences, retain)

In [13]:
del sentences

In [14]:
concated = np.concatenate(smallerSents)

In [15]:
del smallerSents

In [16]:
additional_custom_stopwords = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',,', '[', ']', 'vgl', '...', \
                               'ovg', 'nrw','de',"a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "bzw", \
                               "k", "l", "m", "n", "o", "p", "q", "r","s", "t", "u", "v", "w", "x", "y", "z"]

regexNoSpecialStuff = r"[^A-zäöüß\s]"

In [17]:
for newStop in additional_custom_stopwords:
    if newStop not in nltkStopWords:
        nltkStopWords.append(newStop)

In [18]:
def buildRegexFromList(stringsToMatch):
    init = ""
    for s in stringsToMatch:
        init += r'\b'
        init += re.escape(s)
        init += r'\b|'
    #remove last pipe:
    init = init[:-1]
    return init

In [19]:
regExStopWords = buildRegexFromList(nltkStopWords)

In [20]:
for i, d in enumerate(concated):
    concated[i] = concated[i].lower()
    concated[i] = re.sub(regexNoSpecialStuff, '', d)
    concated[i] = re.sub(regExStopWords, "", d)

In [12]:
with open("cleaned_concated.pkl", "rb") as f:
    concated = pickle.load(f)

In [13]:
nrWords = 3000
vocab_size=nrWords

In [14]:
tokenizer = Tokenizer(num_words=nrWords, oov_token="<OOV>") # nrWords
tokenizer.fit_on_texts(concated)
# determine the vocabulary size
#vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 3000


In [15]:
tokenizer.num_words = nrWords

In [16]:
len(tokenizer.word_counts), tokenizer.num_words, vocab_size

(394112, 3000, 3000)

In [17]:
# smallerSents[0]

In [18]:
# tokenizer.texts_to_sequences(smallerSents)

In [19]:
trainRetain = math.floor(len(concated))

In [20]:
train = np.random.choice(concated, trainRetain)

In [21]:
maxLen = 7

In [22]:
# create line-based sequences
sequences = list()
for encoded in tokenizer.texts_to_sequences(concated):
    if len(encoded) > 1:
        for i in range(0, (len(encoded) - 2) if len(encoded)-2 < maxLen else maxLen):
            sequences.append(encoded[i:i+3])
    
print('Total Sequences: %d' % len(sequences))

Total Sequences: 7942834


In [23]:
sequences = np.array(sequences)
sequences = sequences.astype(np.uint8)

In [24]:
sequences[:, :-1]

array([[233, 174],
       [174, 149],
       [149,  16],
       ...,
       [  4,   1],
       [  1, 142],
       [116,   1]], dtype=uint8)

In [26]:
X = pad_sequences(sequences[:,:-1], maxlen=maxLen)

In [ ]:
y = to_categorical(sequences[:,-1] , num_classes=nrWords,  dtype="uint8") # sequences[:, -1]#

In [ ]:
X.shape, y.shape

In [ ]:
adam = Adam()
reduceLr = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=10)

In [ ]:
# define model
i = tf.keras.layers.Input(shape=(X.shape[1]))
e = tf.keras.layers.Embedding(vocab_size, 5, input_length=maxLen)(i) # , input_length=max_length

l = tf.keras.layers.LSTM(20, return_sequences=False)(e)
# l2 = tf.keras.layers.LSTM(50, return_sequences=False)(l)
# l3 = tf.keras.layers.LSTM(20, return_sequences=True)(l2)
# l4 = tf.keras.layers.LSTM(50, return_sequences=True)(l3)
# l5 = tf.keras.layers.LSTM(20, return_sequences=True)(l4)
# l6 = tf.keras.layers.LSTM(50, return_sequences=True)(l5)
# l7 = tf.keras.layers.LSTM(20, return_sequences=True)(l6)
# l8 = tf.keras.layers.LSTM(50, return_sequences=False)(l7)

# g = tf.keras.layers.GRU(50, return_sequences=True)(l2)
# g2 = tf.keras.layers.GRU(10)(g)

d  = tf.keras.layers.Dense(50, activation="relu")(l)
d2 = tf.keras.layers.Dense(vocab_size, activation='softmax')(d)

model = tf.keras.Model(inputs=i, outputs=[d2])
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# tf.keras.utils.plot_model(model)

In [ ]:
del concated

In [ ]:
model.fit(X, y, epochs=3, verbose=1, callbacks=[reduceLr])

In [82]:
model.fit(X, y, epochs=3, verbose=1, callbacks=[reduceLr])

Epoch 1/3
168546/168546 [==============================] - 655s 4ms/step - loss: 3.5885 - accuracy: 0.3232
Epoch 2/3
168546/168546 [==============================] - 655s 4ms/step - loss: 3.4540 - accuracy: 0.3325
Epoch 3/3
168546/168546 [==============================] - 655s 4ms/step - loss: 3.4331 - accuracy: 0.3342


In [30]:
text = "Der Antrag wird"
text = " ".join(text.split(" ")[:2])
text = text.lower()
text

'der antrag'

In [31]:
next = model.predict(encoded, verbose=0)
for x in next:
    next_word_token = np.argmax(x)
    # map predicted word index to word
    for word, index in tokenizer.word_index.items():
        if index == next_word_token:
            print(word + " ")

<OOV> 
<OOV> 


In [33]:
next = model.predict(encoded) 

In [36]:
for x in next:
    next_word_token = np.argmax(x)
    print(next_word_token)
    tokenizer.word_index.get(next_word_token)

1
1


In [37]:
tokenizer.word_docs

defaultdict(int,
            {'tenor': 9407,
             'einziehung': 483,
             'auf': 11654,
             'angeordnet': 1977,
             'höhe': 30559,
             '50': 10488,
             '€': 21186,
             'soweit': 23657,
             'urteil': 57206,
             '2017': 10108,
             'halle': 868,
             'angeklagten': 6791,
             'aufgehoben': 4280,
             '620': 220,
             'entfällt': 1132,
             'wertersatz': 110,
             'worden': 51500,
             'landgerichts': 4751,
             '20': 32297,
             'übersteigenden': 239,
             '155': 1576,
             'erweiterte': 253,
             'dezember': 20228,
             'taterträgen': 9,
             'revision': 15007,
             'staatskasse': 409,
             'die': 208785,
             'tragen': 6851,
             'gründe': 14354,
             'revisionsverfahren': 1390,
             'kosten': 24553,
             'notwendigen': 2933,
         

In [106]:
# with open("tokenizer.pkl", "wb") as f:
#     pickle.dump(tokenizer, f)

In [107]:
# model.save("./model_1")

In [108]:
# tokenizer.word_counts

In [85]:
tokenizer.word_index.get("tenor", "not in index")

319

In [86]:
encoded = tokenizer.texts_to_sequences([text])
encoded

[[2, 137]]

In [87]:
tokenizer.word_index

{'<OOV>': 1,
 'der': 2,
 'die': 3,
 'des': 4,
 'und': 5,
 'in': 6,
 'nicht': 7,
 'zu': 8,
 'den': 9,
 'von': 10,
 'im': 11,
 'das': 12,
 'ist': 13,
 'dem': 14,
 'auf': 15,
 'vom': 16,
 'für': 17,
 'dass': 18,
 'abs': 19,
 'eine': 20,
 'mit': 21,
 'nach': 22,
 'auch': 23,
 'sich': 24,
 'hat': 25,
 'einer': 26,
 'als': 27,
 'es': 28,
 'ein': 29,
 'aus': 30,
 'bei': 31,
 'zur': 32,
 'an': 33,
 'sei': 34,
 'durch': 35,
 'werden': 36,
 'oder': 37,
 'kläger': 38,
 'sie': 39,
 'klägerin': 40,
 'nr': 41,
 'wird': 42,
 'satz': 43,
 'a': 44,
 'zum': 45,
 'er': 46,
 'sind': 47,
 'vgl': 48,
 'eines': 49,
 'nur': 50,
 'einen': 51,
 'über': 52,
 'habe': 53,
 'beklagte': 54,
 'wenn': 55,
 'beklagten': 56,
 'am': 57,
 'einem': 58,
 'diese': 59,
 'unter': 60,
 'b': 61,
 'kann': 62,
 'vor': 63,
 'urteil': 64,
 'rn': 65,
 's': 66,
 'um': 67,
 'bis': 68,
 'wie': 69,
 'worden': 70,
 'dieser': 71,
 'gegen': 72,
 'keine': 73,
 'art': 74,
 'dies': 75,
 'noch': 76,
 'höhe': 77,
 'ob': 78,
 'war': 79,
 'so': 80